<a href="https://colab.research.google.com/github/Giuseppe86-lab/Affidabilit-Creditizia/blob/main/Affidabilita_creditizia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Esplorazione del dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import RocCurveDisplay
from sklearn.base import clone
from sklearn.model_selection import learning_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold

In [2]:
URL = 'https://proai-datasets.s3.eu-west-3.amazonaws.com/credit_scoring.csv'
df = pd.read_csv(URL, index_col=0)
print(df.shape)
df.head()

(338427, 18)


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET
ID,,,,,,,,,,,,,,,,,,
5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005.0,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,0
5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005.0,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,0
5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474.0,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,0
5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110.0,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1
5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110.0,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1


In questo dataset abbiamo 338427 samples, 17 feature (**ID** è solo il numero identificativo del cliente quindi non contiene informazioni) e una variabile **target**:

1. **CODE_GENDER**: variabile qualitativa categorica che indica il sesso del cliente.
2. **FLAG_OWN_CAR**: variabile qualitativa categorica che indica il possesso di un'automobile.
3. **FLAG_OWN_REALTY**: variabile qualitativa categorica che indica se il cliente possiede una casa.
4. **CNT_CHILDREN**: variabile quantitativa discreta che indica il numero dei figli.
5. **AMT_INCOME_TOTAL**: variabile quantitativa continua del reddito annuale.
6. **NAME_INCOME_TYPE**: variabile qualitativa categorica che indica il tipo di reddito.
7. **NAME_EDUCATION_TYPE**: variabile qualitativa categorica che indica il tipo di formazione del cliente.
8. **NAME_FAMILY_STATUS**: variabile qualitativa categorica che indica lo stato di famiglia del cliente.
9. **NAME_HOUSING_TYPE**: variabile qualitativa categorica che indica il tipo di abitazione.
10. **DAYS_BIRTH**: variabile quantitativa continua che indica i giorni passati dalla nascita (negativa se il cliente è vivo).
11. **DAYS_EMPLOYED**: variabile quantitativa continua che indica i giorni trascorsi dalla data di assunzione, se è positiva indica i giorni da quando il cliente è disoccupato.
12. **FLAG_MOBIL**: variabile qualitativa categorica che indica la presenza di un numero di cellulare.
13. **FLAG_WORK_PHONE**: variabile qualitativa categorica che indica la presenza di un numero di telefono di lavoro.
14. **FLAG_PHONE**: variabile qualitativa categorica che indica la presenza di un numero di telefono.
15. **FLAG_EMAIL**: variabile qualitativa categorica che indica la presenza di un indirizzo mail.
16. **OCCUPATION_TYPE**: variabile qualtitativa categorica che indica il tipo di occupazione.
17. **CNT_FAM_MEMBERS**: variabile quantitativa discreta che indica il numero di familiari.
18. **TARGET**: variabile qualitativa ordinale che vale 1 se il client eha una elevata affidabilità creditizia (pagamento costante delle rate), 0 altrimenti.

L'obiettivo del progetto è sviluppare un **modello di machine learning** che preveda se i clienti hanno una buona affidabilità creditizia.

## 1.1 Caratteristiche generali del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 338427 entries, 5008804 to 6392180
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CODE_GENDER          338427 non-null  object 
 1   FLAG_OWN_CAR         338427 non-null  object 
 2   FLAG_OWN_REALTY      338427 non-null  object 
 3   CNT_CHILDREN         338427 non-null  int64  
 4   AMT_INCOME_TOTAL     338427 non-null  float64
 5   NAME_INCOME_TYPE     338427 non-null  object 
 6   NAME_EDUCATION_TYPE  338427 non-null  object 
 7   NAME_FAMILY_STATUS   338426 non-null  object 
 8   NAME_HOUSING_TYPE    338426 non-null  object 
 9   DAYS_BIRTH           338426 non-null  float64
 10  DAYS_EMPLOYED        338426 non-null  float64
 11  FLAG_MOBIL           338426 non-null  float64
 12  FLAG_WORK_PHONE      338426 non-null  float64
 13  FLAG_PHONE           338426 non-null  float64
 14  FLAG_EMAIL           338426 non-null  float64
 15  OCCUPATION_TYPE

Nel nostro dataset sono presenti feature di vario tipo:

- **int**: Target
- **float**: AMT_INCOME_TOTAL, DAYS_BIRTH, DAYS_EMPLOYED, FLAG_MOBIL, FLAG_WORK_PHONE, FLAG_PHONE, FLAG_EMAIL, CNT_FAM_MEMBERS
- **object**: CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE

È importante andare a osservare il tipo delle feature perché in base alle loro caratteristiche e tipologie dovremmo svolgere delle diverse analisi esplorative. Le informazioni ottenute con *df.info()* ci dicono anche che nel nostro dataset ci sono valori mancanti.

In [6]:
df.isnull().sum(axis=0)

,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,1
NAME_HOUSING_TYPE,1
DAYS_BIRTH,1


Possiamo osservare che per circa un terzo dei sample del nostro dataset non è presente l'informazione su la feature **OCCUPATION_TYPE**, anche in altre feature ci sono valori mancanti ma un numero irrisori, invece per questa dovrebbe valutare la migliore strategia da adottare in fare di preprocessing. Vogliamo verificare se per caso tutti quei valori mancanti non appartengano ad uno stesso sample:

In [11]:
df[df['NAME_FAMILY_STATUS'].isna()]

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TARGET
ID,,,,,,,,,,,,,,,,,,
6392180,F,N,N,0,67500.0,Working,Secondary / se,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Effettivamente come sospettavamo tutti i NaN si concentrano in uno stesso sample che quindi possiamo decidere di eliminare.

In [13]:
df = df.dropna(subset=['NAME_FAMILY_STATUS'])
df.isnull().sum(axis=0)

,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0
DAYS_BIRTH,0


Verifichiamo se ci sono samples ripetuti:

In [16]:
duplicate_rows = df.duplicated()

duplicates = df[df.duplicated()]
print(duplicates)
df.shape

        CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
ID                                                               
5008805           M            Y               Y             0   
5008809           F            N               Y             0   
5008810           F            N               Y             0   
5008811           F            N               Y             0   
5008813           F            N               Y             0   
...             ...          ...             ...           ...   
6392174           F            N               Y             0   
6392175           F            N               Y             0   
6392177           F            N               Y             0   
6392178           F            N               Y             0   
6392179           F            N               Y             0   

         AMT_INCOME_TOTAL      NAME_INCOME_TYPE  \
ID                                                
5008805          427500.0              

(338426, 18)

Nel nostro dataset c'è il 78% di valori duplicati che non portano informazioni aggiuntive al nostro modello, quindi possiamo eliminarli.

In [21]:
df= df.drop_duplicates()
print(df.shape)
df.isnull().sum(axis=0)

(75256, 18)


,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0
DAYS_BIRTH,0


Eliminando i sample duplicati i valori mancanti di **OCCUPATION_TYPE** si sono ridotti proporzionalmente.

In [19]:
df[['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']].describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
count,75256.000000,7.525600e+04,75256.000000,75256.000000,75256.000000
mean,0.409070,1.848544e+05,-16076.620695,56591.817689,2.165874
std,0.719442,1.069342e+05,4185.574252,135218.584928,0.896137
min,0.000000,2.610000e+04,-25201.000000,-17531.000000,1.000000
25%,0.000000,1.125000e+05,-19482.000000,-3124.000000,2.000000
50%,0.000000,1.575000e+05,-15969.500000,-1493.000000,2.000000
75%,1.000000,2.250000e+05,-12600.750000,-398.000000,3.000000
max,19.000000,6.750000e+06,-7489.000000,365243.000000,20.000000


Ciò che salta subito all'occhio è che il 75% dei clienti ha al massimo 1 figlio ma c'è un valore anomale per un cliente con 19 figli.

In [22]:
df.describe(include='object')

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE
count,75256,75256,75256,75256,75256,75256,75256,53131
unique,2,2,2,5,5,5,6,18
top,F,N,Y,Working,Secondary / secondary special,Married,House / apartment,Laborers
freq,49005,47677,51376,39178,52714,50239,67360,13955


Vediamo quali sono i valori assunti da **NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE**:

In [24]:
columns = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']

for column in columns:
    print(f'{column}: {df[column].unique()} \n')

NAME_INCOME_TYPE: ['Working' 'Commercial associate' 'Pensioner' 'State servant' 'Student'] 

NAME_EDUCATION_TYPE: ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree'] 

NAME_FAMILY_STATUS: ['Civil marriage' 'Married' 'Single / not married' 'Separated' 'Widow'] 

NAME_HOUSING_TYPE: ['Rented apartment' 'House / apartment' 'Municipal apartment'
 'With parents' 'Co-op apartment' 'Office apartment'] 

OCCUPATION_TYPE: [nan 'Security staff' 'Sales staff' 'Accountants' 'Laborers' 'Managers'
 'Drivers' 'Core staff' 'High skill tech staff' 'Cleaning staff'
 'Private service staff' 'Cooking staff' 'Low-skill Laborers'
 'Medicine staff' 'Secretaries' 'Waiters/barmen staff' 'HR staff'
 'Realty agents' 'IT staff'] 



## 1.2 Analisi Univariata

In questa fase andiamo a studiare le distribuzioni delle singole feature.

Per le variabile categoriche realizziamo dei grafici a barre per vedere la distribuzione delle diverse classi, se sono o meno equamente rappresentate.